# Lua examples
We are using Redis Server 6

This allows us to use RESP3 in our lua scripts.  
The python module hasn't started to implement RESP3 but that is fine for us.

Coming from python table access like table['key'] = value comes very handy

For the start the lua scripts will be more in the context of building views from our core redis storage. So they can be slightly complex.

In [ ]:
import datetime as dt
import pandas as pd
import numpy as np
import redis

from fastcounting import helper, views, store, files, system

r = redis.Redis(**helper.Helper().rediscred, decode_responses=True)

# Unique list of all accounts where we have action
We achieve this by iterating over all atomic transactions in the 'account:atomic'.
The RESP3 lua data we get from ZRANGEBYSCORE looks like this:

{{"171255"; {["double"]=135}}; {"75069"; {["double"]=135}}; {"169292"; {["double"]=650}};

In [ ]:
lua_accounts = """
redis.setresp(3)
local accounts = {}
local hash = {}
for i, value in pairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2], ARGV[3])) do
    local account = value[2]['double']
    if not hash[account] then
        accounts[#accounts+1] = account
        hash[account] = true
    end
end
if ARGV[4] == nil then return accounts end
"""
response = r.eval(lua_accounts, 1, 'account:atomic', 0, 9999999, 'WITHSCORES')

# Sum up all accounts and return a 1 dimensional list to python RESP2 client

If the python client would support resp3 we would be finished after the first for loop.

First i looped over all values with scores (account number) in 'account:atomic'  
I made a unique array from all the scores or from the range of scores you specified  
In our case we have two more for loops. Second we create 2 lists, than we concat those 2 lists. That should be fine for now.  

In [ ]:
lua_sum = """
redis.setresp(3)
local result = {}
local hash = {}
for i, value in pairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2], ARGV[3])) do
    local account = value[2]['double']
    local atomic = value[1]
    if result[account] == nil then result[account] = 0 end
    result[account] = result[account] + redis.call('HGET', 'atomicID:' .. atomic, 'amount')
end

local xaccounts = {}
local xsums = {}
for xaccount, xsum in pairs(result) do
    xaccounts[#xaccounts+1] = xaccount
    xsums[#xsums+1] = xsum
end

for i=1, #xsums do
    xaccounts[#xaccounts + 1] = xsums[i]
end
return xaccounts
"""

In [ ]:
response = r.eval(lua_sum, 1, 'account:atomic', 0, 9999999, 'WITHSCORES')

# Create views for all accounts, with the help of redis streams

For views we always use 'atomic:date' as entrypoint. So we can add all atomics to our date aware redis streams (stack like).

In [ ]:
lua_stable_view = """
redis.setresp(3)
for i, atomicID in pairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2])) do

    local atomic = redis.call('HGETALL', 'atomicID:' .. atomicID)['map']
    local general = redis.call('HGETALL', 'generalID:' .. atomic['generalID'])['map']
    local account = redis.call('HGETALL', 'accountsystem:' .. atomic['accountID'])['map']
    if next(account)==nil then
        account = redis.call('HGETALL', 'accountsystem:special_account')['map'] end
    
    redis.call('XADD','account:' .. atomic['accountID'],
        general['date'] .. '-' .. i, 
        'general', atomic['generalID'])
end
"""
# arguments are dates, we don't need accounts because we create a stream for every account
response = r.eval(lua_stable_view, 1, 'atomic:date', 0, dt.datetime(2018, 1, 2).timestamp())

Delete account views

In [ ]:
delete_stable_views = lua_accounts + """
for i, account in pairs(accounts) do
    redis.call('DEL', 'account:' .. account)
end
return true
"""
response = r.eval(delete_stable_views, 1, 'account:atomic', 0, 9999999, 'WITHSCORES', 'dont')

In [ ]:
# show accountviews
r.xrevrange('account:1830', 1513728000, 0)

# Create atomicview 
Use case: Get all atomics for your defined timespan

In [ ]:
lua_atomic_view = """
redis.setresp(3)
for i, atomicID in pairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2])) do
    local atomic = redis.call('HGETALL', 'atomicID:' .. atomicID)['map']
    local general = redis.call('HGETALL', 'generalID:' .. atomic['generalID'])['map']
    local account = redis.call('HGETALL', 'accountsystem:' .. atomic['accountID'])['map']
    if next(account)==nil then
        account = redis.call('HGETALL', 'accountsystem:special_account')['map'] end
    
    redis.call('XADD','atomicview',
        general['date'] .. '-' .. i, 
        'general', atomic['generalID'])
end
return true
"""
response = r.eval(lua_atomic_view, 1, 'atomic:date', 0, dt.datetime(2018, 1, 2).timestamp())

In [ ]:
# show atomicview
response = r.xrevrange(
    'atomicview',
    int(dt.datetime(2018, 1, 2).timestamp()),
    int(dt.datetime(2017, 1, 1).timestamp()),
    count=2000000)

In [ ]:
# delete atomic view -> we have only one without multistates
r.delete('atomicview')

parse response to a pandas Dataframe

In [ ]:
data = np.array(response).flatten()
df = pd.DataFrame(data[1::2].tolist(), index=data[0::2])
df